In [2]:
import os

import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import display

import torch
from torch import nn
import torchsort

In [3]:
train_year = 2018
train_stock = {}
test_stock = {}

folder_pth = os.readlink("data/symlink")

for idx, path in tqdm(enumerate(glob(f"{folder_pth}/*.csv"))):
    curr_df = pd.read_csv(path)
    stock_name = path.split('/')[-1].split('.')[0]
    
    train_df = curr_df.loc[curr_df['date'].apply(lambda x: int(x.split('-')[0]) < train_year)]
    test_df = curr_df.loc[curr_df['date'].apply(lambda x: int(x.split('-')[0]) >= train_year)]
    
    if len(train_df):
        train_stock[stock_name] = train_df
    
    if len(test_df):
        test_stock[stock_name] = test_df

1087it [00:07, 147.78it/s]


In [4]:
subset_stock = ['2330']

subset_train_stock = train_stock #{_:train_stock[_] for _ in subset_stock}
subset_test_stock = test_stock #{_:test_stock[_] for _ in subset_stock}

In [5]:
train_stock[list(train_stock.keys())[10]]

,date,證券代號,開盤價,最高價,最低價,收盤價,成交股數,adj_close
0,2007-04-23,9939,31.00,31.60,30.70,30.80,3818579.0,32.757224
1,2007-04-24,9939,30.80,31.10,30.00,30.60,5984851.0,32.544515
2,2007-04-25,9939,30.60,30.60,29.80,30.00,4290668.0,31.906387
3,2007-04-26,9939,30.25,30.25,29.55,29.75,3748365.0,31.640500
4,2007-04-27,9939,29.75,29.80,29.35,29.35,2315366.0,31.215082
...,...,...,...,...,...,...,...,...
2647,2017-12-25,9939,57.50,57.80,57.30,57.50,351603.0,103.170459
2648,2017-12-26,9939,57.40,57.40,56.70,56.70,723297.0,101.735044
2649,2017-12-27,9939,56.70,57.10,56.70,57.00,299195.0,102.273325
2650,2017-12-28,9939,57.00,57.40,56.50,57.00,348260.0,102.273325


In [6]:
past_window, future_window = 60, 20
full_window = past_window + future_window
buy_fee = 0.001425
sell_fee = 0.003 + buy_fee

data = {}

for curr_phase, curr_data in tqdm({'train': subset_train_stock, 'test': subset_test_stock}.items()):
    data[curr_phase] = {'x': [], 'y': [], 'y_with_cost': [], 'name': [], 'date': []}

    for idx, (stock_name, stock_df) in enumerate(curr_data.items()):
        close_array = stock_df['adj_close'].to_numpy()
        volume_array = stock_df['成交股數'].to_numpy()
        date_array = stock_df['date'].to_numpy()

        window_index = np.arange(
            close_array.shape[0] - full_window + 1
        ).reshape(-1, 1) + np.arange(full_window)

        close_window_array = close_array[window_index]
        volume_window_array = volume_array[window_index]
        date_window_array = date_array[window_index]

        close_anchor = close_window_array[:, 1:past_window]
        volume_anchor = (
            np.log(volume_window_array[:, 1:past_window] + 1) - np.log(volume_window_array[:, :1]) + 1
        ) / np.log(volume_window_array[:, :1] + 1)
        date_anchor = date_window_array[:, -1]

        raw_leave = (
            close_window_array[:, -future_window: ] - close_window_array[:, past_window - 1: past_window]
        ) / close_window_array[:, past_window - 1: past_window]
        sell_price = close_window_array[:, -future_window:] * (1 - sell_fee)
        buy_price = close_window_array[:, past_window-1:past_window] * (1 + buy_fee)
        real_leave = (sell_price - buy_price) / buy_price

        data[curr_phase]['x'].append(close_anchor.reshape(close_anchor.shape[0], close_anchor.shape[1], 1))
        data[curr_phase]['y'].append(raw_leave)
        data[curr_phase]['y_with_cost'].append(real_leave)
        data[curr_phase]['name'].append(np.array([stock_name]*close_anchor.shape[0]))
        data[curr_phase]['date'].append(date_anchor)

    data[curr_phase]['x'] = np.concatenate(data[curr_phase]['x'])
    data[curr_phase]['y'] = np.concatenate(data[curr_phase]['y'])
    data[curr_phase]['y_with_cost'] = np.concatenate(data[curr_phase]['y_with_cost'])
    data[curr_phase]['name'] = np.concatenate(data[curr_phase]['name'])
    data[curr_phase]['date'] = np.concatenate(data[curr_phase]['date'])

100%|██████████| 2/2 [00:07<00:00,  3.95s/it]


In [7]:
data["test"]['x'].shape

(1563611, 59, 1)

In [8]:
def differentiable_spearman(pred, target):
    pred_rank = torchsort.soft_rank(pred)
    target_rank = torchsort.soft_rank(target)
    pred_n = pred_rank - pred_rank.mean()
    target_n = target_rank - target_rank.mean()
    pred_n = pred_n / pred_n.norm()
    target_n = target_n / target_n.norm()
    corr = (pred_n * target_n).sum()
    return corr


class MAMA(nn.Module):
    def __init__(self, max_history=30, n_alpha=10):
        super().__init__()
        
        self.n_alpha = n_alpha
        self.alpha_linear = nn.Linear(max_history, n_alpha * 2, bias=False)
        
    def forward(self, x):
        alpha_weight = self.alpha_linear.weight.softmax(-1)
        alpha = nn.functional.linear(x, alpha_weight)
        alpha = (alpha[:, :self.n_alpha] - alpha[:, self.n_alpha:]).tanh()
        return alpha

In [9]:
tr_x = []
tr_y = []
te_x = []
te_y = []

for i, stock_name in tqdm(enumerate(train_stock.keys())):
    tr_x.append(data['train']['x'][data['train']['name'] == stock_name].squeeze().astype('float32'))
    tr_y.append(data['train']['y_with_cost'][data['train']['name'] == stock_name, -1].astype('float32'))
    te_x.append(data['test']['x'][data['test']['name'] == stock_name].squeeze().astype('float32'))
    te_y.append(data['test']['y_with_cost'][data['test']['name'] == stock_name, -1].astype('float32'))

969it [01:05, 14.72it/s]


In [16]:
tr_x[-1].shape, tr_y[-1].shape

((862, 59), (862,))

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MAMA(max_history=past_window-1, n_alpha=1).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.0)

In [10]:
tr_x[0].shape

(2613, 19)

In [14]:
for epoch in range(1000):
    
    epoch_loss = []
    
    for stock_tr_x, stock_tr_y in zip(tr_x, tr_y):
    
        out = model(torch.tensor(stock_tr_x, dtype=torch.float32).to(device))

        loss = 0
        for _ in range(out.shape[-1]):
            curr_loss = -differentiable_spearman(
                out[:, _:_+1].t(),
                torch.tensor(stock_tr_y, dtype=torch.float32).unsqueeze(0).to(device)
            )
            loss += curr_loss
            epoch_loss.append(loss.item())
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(np.mean(epoch_loss))

-0.030494062505831598
-0.03555363092955463
-0.03953002655276283
-0.04253687478060823
-0.044885372572892775
-0.04684892512718072
-0.048544764311617035
-0.05007038740835261
-0.05138449727195259
-0.052442589964067914
-0.05329685356449701
-0.053974708052959015
-0.054493584344187565
-0.05491284498275941
-0.05522666905619304
-0.05548755900710398
-0.05570324485683266
-0.05588955406142328
-0.05602964786408994
-0.05613334865082955
-0.05621546815628603
-0.0562834687612329
-0.05634091786645695
-0.056390441177553194
-0.05643394590583481
-0.05647289111012001
-0.056507276391965636
-0.056537071221904066
-0.05656325480873313
-0.05658702393626108
-0.05660959760782427
-0.0566299157008102
-0.05664713713566324
-0.05666229861665429
-0.056675953624392324
-0.056688444142160656
-0.05669931143724578
-0.056709038684653046
-0.05671798906054048
-0.05672633739020599
-0.05673415867539201
-0.05674158992143434
-0.05674902935463347
-0.056756307984033354
-0.05676317159545399
-0.056769580208101494
-0.05677552079607447
-

KeyboardInterrupt: 

In [12]:
out.dtype

torch.float32

In [13]:
model.eval()

with torch.no_grad():
    tr_out = model(tr_x.to(device)).cpu()
    te_out = model(te_x.to(device)).cpu()

AttributeError: 'list' object has no attribute 'to'

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(tr_y[:100])
plt.plot(tr_out[:100].sum(-1))
plt.plot(tr_out[:100])

In [ ]:
plt.hist2d(te_out.squeeze().numpy(), te_y.numpy(), bins=40)

In [ ]:
for _ in [-1, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    print(_, ((te_y[te_out.sum(-1) > _] > 0) * 1.).mean())

In [ ]:
te_out.shape

In [ ]:
te_y[te_out[:, 0] > 0.4]

In [ ]:
((te_y[te_out[:, 0] > 0.1] > 0) * 1.).mean()

In [ ]:
((te_y[te_out[:, 0] > -1] > 0) * 1.).mean()

In [ ]:
te_y[te_out[:, 0] < -0.5].mean()

In [ ]:
te_y.mean()

In [ ]:
plt.plot(te_y)
plt.plot(te_out.sum(-1))

In [ ]:
model.alpha_linear.weight[0]

In [ ]:
model.alpha_linear.weight[1]

In [ ]:
(model.alpha_linear.weight[0]).softmax(-1)

In [ ]:
(model.alpha_linear.weight[2]).softmax(-1)

In [ ]:
(model.alpha_linear.weight[1] / model.alpha_temp[1]).softmax(-1)

In [ ]:
a = torch.rand(4, 30)

In [ ]:
wsl = nn.Linear(30, 10, bias=False)
temp = nn.Parameter(torch.ones(10, 1)*0.1)

In [ ]:
wsl.weight.shape

In [ ]:
o = nn.functional.linear(a, wsl.weight)
o

In [ ]:
o = nn.functional.linear(a, (wsl.weight / temp).softmax(-1))
o

In [ ]:
o

In [ ]:
o[:, 5:] - o[:, :5]

In [ ]:
a.shape

In [ ]:
ma 上下穿

In [ ]:
today - weighted_sum_of_history

In [ ]:
train_stock['2330']['成交股數']

In [ ]:
train_stock['2330']['收盤價'][:20]